In [1]:
!pip install streamlit pyngrok groq pymupdf easyocr opencv-python-headless numpy pandas==2.2.2



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.2/978.2 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 13.9 MB/s eta 0:00:00


In [2]:
from pyngrok import ngrok
ngrok.set_auth_token("30RqzYnPrzX8X1QHkU0iHjSA7gq_76gudD9omcU24DVuiC83C")


In [3]:
%%writefile app.py
import streamlit as st
from groq import Groq
import fitz
import pandas as pd
import json
import easyocr
import numpy as np
import cv2

st.set_page_config(page_title="AI Answer Evaluator", layout="wide")
st.title("📘 AI Exam Answer Evaluator Dashboard (Teacher + Student OCR Supported)")

# Sidebar API Key
GROQ_API_KEY = st.sidebar.text_input("Enter Groq API Key", type="password")

if GROQ_API_KEY:
    client = Groq(api_key=GROQ_API_KEY)

# EasyOCR Reader
reader = easyocr.Reader(['en'])

# ---------- SAFE JSON FUNCTION ----------
def safe_json_loads(text):
    try:
        return json.loads(text)
    except:
        start = text.find("{")
        end = text.rfind("}")
        if start != -1 and end != -1:
            try:
                return json.loads(text[start:end+1])
            except:
                return {"error": "JSON Decode Failed", "raw_output": text}
        return {"error": "JSON Decode Failed", "raw_output": text}

# ---------- OCR TEXT EXTRACTION FUNCTION ----------
def extract_text_from_ocr_pdf(pdf_file):
    text = ""
    doc = fitz.open(stream=pdf_file.read(), filetype="pdf")

    for page_num in range(len(doc)):
        page = doc[page_num]
        pix = page.get_pixmap(dpi=300)

        img = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.height, pix.width, pix.n)

        if pix.n == 4:
            img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)

        result = reader.readtext(img, detail=0)
        page_text = "\n".join(result)

        text += f"\n--- Page {page_num+1} ---\n"
        text += page_text

    return text.strip()

# ---------- GROQ CHAT FUNCTION ----------
def groq_chat(prompt):
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
        max_tokens=1500
    )
    return response.choices[0].message.content.strip()

# ---------- EXTRACT QUESTIONS + IDEAL ANSWERS ----------
def extract_questions_and_answers(teacher_text):
    prompt = f"""
You are extracting exam questions and ideal answers from a teacher question paper.

Return ONLY JSON:
{{
  "questions":[
    {{
      "question_no": 1,
      "question": "...",
      "ideal_answer": "...",
      "marks": 2
    }}
  ]
}}

Rules:
- Extract real questions only.
- Extract marks correctly.
- If marks not visible, assume marks = 2.
- Do NOT generate random questions.
- Do NOT add extra text.

TEXT:
{teacher_text}
"""
    return safe_json_loads(groq_chat(prompt))

# ---------- EXTRACT STUDENT ANSWERS ----------
def extract_student_answers(student_text):
    prompt = f"""
You are extracting student answers from OCR text.

Return ONLY JSON:
{{
  "answers":[
    {{
      "question_no": 1,
      "student_answer": "..."
    }}
  ]
}}

Rules:
- Extract answers based on question number.
- OCR text may have spelling mistakes.
- If answer not found, skip it.
- Do NOT generate random answers.
- Do NOT add extra text.

TEXT:
{student_text}
"""
    return safe_json_loads(groq_chat(prompt))

# ---------- EVALUATE ANSWER ----------
def evaluate_answer(question, ideal_answer, student_answer, marks):
    prompt = f"""
You are a strict teacher evaluating student answers.

QUESTION:
{question}

IDEAL ANSWER:
{ideal_answer}

STUDENT ANSWER:
{student_answer}

TOTAL MARKS: {marks}

Rules:
- Fully correct -> full marks
- Partially correct -> partial marks
- Wrong/irrelevant -> 0 marks

Feedback Rules:
- If full marks: include "Excellent" or "Very good"
- If partial marks >= 50%: include "Good attempt" and "Need to improve"
- If partial marks < 50%: include "Need improvement" and "Learn the basics"
- If obtained = 0: include "Incorrect" and "Learn the basics"

Return ONLY JSON:
{{
  "obtained": <0 to {marks}>,
  "feedback": "<feedback>",
  "missing_points": "<missing points or NA>"
}}
"""
    return safe_json_loads(groq_chat(prompt))

# ---------- FILE UPLOAD ----------
teacher_pdf = st.sidebar.file_uploader("📌 Upload Teacher Question Paper PDF", type=["pdf"])
student_pdf = st.sidebar.file_uploader("📌 Upload Student Answer Sheet PDF", type=["pdf"])

if st.sidebar.button("Evaluate"):
    if not GROQ_API_KEY:
        st.error("❌ Enter Groq API Key")
    elif not teacher_pdf or not student_pdf:
        st.error("❌ Upload both Teacher and Student PDFs")
    else:
        st.info("⏳ OCR Scanning Teacher PDF...")
        teacher_text = extract_text_from_ocr_pdf(teacher_pdf)

        st.info("⏳ OCR Scanning Student PDF...")
        student_text = extract_text_from_ocr_pdf(student_pdf)

        # Show extracted OCR text
        st.subheader("📌 Extracted Teacher OCR Text (First 5000 chars)")
        st.text_area("Teacher OCR Extract", teacher_text[:5000], height=250)

        st.subheader("📌 Extracted Student OCR Text (First 5000 chars)")
        st.text_area("Student OCR Extract", student_text[:5000], height=250)

        # Extract questions
        st.info("⏳ Extracting Questions + Ideal Answers...")
        qa_data = extract_questions_and_answers(teacher_text)

        if "error" in qa_data:
            st.error("❌ Teacher PDF question extraction failed!")
            st.write(qa_data)
            st.stop()

        # Extract student answers
        st.info("⏳ Extracting Student Answers...")
        student_data = extract_student_answers(student_text)

        if "error" in student_data:
            st.error("❌ Student answer extraction failed!")
            st.write(student_data)
            st.stop()

        questions = qa_data.get("questions", [])
        student_answers = {a["question_no"]: a["student_answer"] for a in student_data.get("answers", [])}

        if len(questions) == 0:
            st.error("❌ No questions detected in teacher PDF. Please upload correct file.")
            st.stop()

        results = []
        total_marks = 0
        obtained_marks = 0

        st.success("✅ Questions Extracted Successfully!")

        # Evaluate each question
        for q in questions:
            qno = q.get("question_no")
            question = q.get("question", "")
            ideal = q.get("ideal_answer", "")
            marks = q.get("marks", 2)

            student_ans = student_answers.get(qno, "No Answer Provided")

            eval_data = evaluate_answer(question, ideal, student_ans, marks)

            if "error" in eval_data:
                st.error(f"❌ Evaluation failed for Question {qno}")
                st.write(eval_data)
                continue

            total_marks += marks
            obtained_marks += eval_data.get("obtained", 0)

            results.append({
                "Q.No": qno,
                "Question": question,
                "Student Answer": student_ans,
                "Obtained": eval_data.get("obtained", 0),
                "Total": marks,
                "Missing Points": eval_data.get("missing_points", "NA"),
                "Feedback": eval_data.get("feedback", "NA")
            })

        df = pd.DataFrame(results)

        # Final Score
        st.success("✅ Evaluation Completed!")
        st.markdown(f"## 🎯 Final Score: **{obtained_marks}/{total_marks}**")

        percentage = (obtained_marks / total_marks) * 100 if total_marks > 0 else 0
        st.markdown(f"### 📊 Percentage: **{percentage:.2f}%**")

        # Grade Calculation
        if percentage >= 90:
            grade = "A+"
        elif percentage >= 80:
            grade = "A"
        elif percentage >= 70:
            grade = "B"
        elif percentage >= 60:
            grade = "C"
        elif percentage >= 50:
            grade = "D"
        else:
            grade = "Fail"

        st.markdown(f"### 🏅 Grade: **{grade}**")

        st.subheader("📌 Detailed Evaluation Report")
        st.dataframe(df, use_container_width=True)

        st.download_button(
            "⬇ Download Evaluation Report CSV",
            df.to_csv(index=False),
            "evaluation_report.csv"
        )


Writing app.py


In [4]:
!pip install streamlit pyngrok groq pymupdf easyocr opencv-python-headless numpy pandas==2.2.2


In [5]:
!streamlit run app.py --server.port 8501 --server.address 0.0.0.0 &>/content/logs.txt &



In [6]:
from pyngrok import ngrok
public_url = ngrok.connect(8501)
print("✅ Dashboard Link:", public_url)


✅ Dashboard Link: NgrokTunnel: "https://a665-34-145-130-71.ngrok-free.app" -> "http://localhost:8501"
